In [2]:
import pandas as pd
import numpy as np
import time

import db_operations as dbop
import data_process as dp
import constants as const
from script import *
import ml_model
import indicators

In [3]:
cursor = dbop.connect_db("sqlite3").cursor()
start = 20130101
df = dbop.create_df(cursor, const.STOCK_DAY[const.TABLE],
                        start=start,
                        )
df = dp.prepare_each_stock(dp.prepare_stock_d(df))
print(df.shape)
print(df.head())

select * from stock_day where date>=20130101


(1982974,)
(1982974, 6)


(1982974, 14)
                           open       high        low      close  \
code      date                                                     
000001.SZ 2013-01-04  590.34336  595.04585  575.87416  578.40627   
          2013-01-07  578.04454  591.42855  574.42724  589.61990   
          2013-01-08  589.61990  592.15201  573.70378  578.76800   
          2013-01-09  577.32108  579.49146  571.53340  573.70378   
          2013-01-10  573.70378  582.38530  571.17167  574.06551   

                               vol          amt  adj_factor         avg  \
code      date                                                            
000001.SZ 2013-01-04  12270.239405  717567.5466      36.173  584.803216   
          2013-01-07   9873.918392  578450.4876      36.173  585.836812   
          2013-01-08   8638.462942  501360.0937      36.173  580.381136   
          2013-01-09   6947.976391  399696.1831      36.173  575.269921   
          2013-01-10   6635.619661  383347.7326      36.173

In [58]:
cursor = dbop.connect_db("sqlite3").cursor()
start = 20130101
df = dbop.create_df(cursor, const.STOCK_DAY[const.TABLE],
                        start=start,
#                         where_clause="code='002349.SZ'"
                        )
df = dp.prepare_stock_d(df)
print(df.shape)
print(df.index.is_lexsorted())

select * from stock_day where date>='20140101'
                 0         1      2      3      4      5           6  \
0        600401.SH  20180529   0.87   0.87   0.87   0.87        0.00   
1        600401.SH  20180530   0.87   0.87   0.87   0.87        0.00   
2        600401.SH  20180531   0.87   0.87   0.87   0.87        0.00   
3        600401.SH  20180601   0.87   0.87   0.87   0.87        0.00   
4        600401.SH  20180604   0.87   0.87   0.87   0.87        0.00   
5        600401.SH  20180605   0.87   0.87   0.87   0.87        0.00   
6        600401.SH  20180606   0.87   0.87   0.87   0.87        0.00   
7        600401.SH  20180607   0.87   0.87   0.87   0.87        0.00   
8        600401.SH  20180608   0.87   0.87   0.87   0.87        0.00   
9        600401.SH  20180611   0.87   0.87   0.87   0.87        0.00   
10       600401.SH  20180612   0.87   0.87   0.87   0.87        0.00   
11       600401.SH  20180613   0.87   0.87   0.87   0.87        0.00   
12       600401.S

In [ ]:
import os
dir = r"database"
f_name1 = "return_5%_25%_20_20"
f_name2 = "return_10%_25%_20_20"
r1 = pd.read_parquet(os.path.join(dir,f_name1))
r2 = pd.read_parquet(os.path.join(dir,f_name2))
print(r1.index.is_lexsorted(),r2.index.is_lexsorted())
print(r1.shape,r2.shape)
if not r1.index.is_lexsorted():
    r1.sort_index(inplace=True)
if not r2.index.is_lexsorted():
    r2.sort_index(inplace=True)
print(r1.index.is_lexsorted(),r2.index.is_lexsorted())

TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:

r1["r"] = (r1["sell_price"]/r1["open"]-1)*100
r2["r"] = (r2["sell_price"]/r2["open"]-1)*100


TypeError: slice indices must be integers or None or have an __index__ method

In [21]:

print(r1["r"].head(),r2["r"].head())

TypeError: slice indices must be integers or None or have an __index__ method

In [24]:

# r_names = [("r1",r1),("r2",r2)]
def stock_month_analysis(r):
    column_ops = [("mean",np.mean,{}),("median",np.nanmedian,{}),
               ("25%",np.nanpercentile,{"q":25}),("75%",np.nanpercentile,{"q":75}),
               ("std",np.std,{})]
    result = pd.DataFrame(columns=[col for col,_,_ in column_ops])
    for i in range(1,13):
        mask = r.index.get_level_values("date").month==i
        row = {col:op(r.loc[mask,"r"],**kwargs) for col,op,kwargs in column_ops}
        result.loc[i] = row
    print(result.to_string())
    return result


TypeError: slice indices must be integers or None or have an __index__ method

In [26]:
stock_month_analysis(r1)
stock_month_analysis(r2)

TypeError: slice indices must be integers or None or have an __index__ method

TypeError: slice indices must be integers or None or have an __index__ method

In [61]:
result_list= []
return_list = []
for i,(code, group) in enumerate(df.groupby("code")):
    if i%50>0:
        continue
    print(code)
    t0 = time.time()
    result_list.append(simple_amt_weighted_MACD(group))
    print(time.time()-t0)
    return_list.append(get_return_rate(group))
    print(time.time()-t0)
# result = simple_amt_weighted_MACD(df)
# pd.set_option("display.max_rows",500)
# print(result)
# t0 = time.time()
# y1 = get_return_rate(df)
# print(time.time()-t0)

# t0 = time.time()
# y2 = get_return_rate2(df)
# print(time.time()-t0)
result = pd.concat(result_list).reset_index()
r = pd.concat(return_list)
r = r.reset_index().merge(df,on=["date","code"],how="left")
df_r = r[["date","code","open"]].copy()
df_r["r"] = (r["r"]/r["open"]-1)*100
print(r.shape,result.shape)
df_all = result.merge(df_r,on=["date","code"],how="inner")
print(df_all.shape)
result = df_all[df_all.columns.difference(["code","date","r"])]
r = df_all[["r"]]
print(result.shape,r.shape)
print(result.columns,r.columns)

(31896, 10) (31896, 9)
(31896, 11)
(31896, 8) (31896, 1)
Index(['dea', 'dif', 'ma12', 'ma26', 'macd', 'macd_pct', 'macd_pct2', 'open'], dtype='object') Index(['r'], dtype='object')


In [62]:
# result["macd_pct2"] = result["macd"]/result["ma26"]
print(y1.index)
print(result.index)
print(result.shape,result.dropna().shape)
print(result.index.difference(y1.index))
# ml_model.corr(result,r[["r"]])

Int64Index([20140102, 20140103, 20140106, 20140107, 20140108, 20140109,
            20140110, 20140113, 20140114, 20140115,
            ...
            20190417, 20190418, 20190419, 20190422, 20190423, 20190424,
            20190425, 20190426, 20190429, 20190430],
           dtype='int64', name='date', length=1279)
Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            31886, 31887, 31888, 31889, 31890, 31891, 31892, 31893, 31894,
            31895],
           dtype='int64', length=31896)
(31896, 8) (30939, 8)
Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            31886, 31887, 31888, 31889, 31890, 31891, 31892, 31893, 31894,
            31895],
           dtype='int64', length=31896)


In [63]:
ml_model.corr(result,r)

30910
(30910, 8) (30910, 1)


,r
dea,0.013195
dif,0.015577
ma12,-0.085524
ma26,-0.087511
macd,-0.012720
macd_pct,-0.001574
macd_pct2,-0.005582
open,-0.085770


In [53]:
print(result)

           dea       dif       ma12       ma26      macd     macd_pct  \
0          NaN       NaN        NaN        NaN       NaN          NaN   
1          NaN       NaN        NaN        NaN       NaN          NaN   
2          NaN       NaN        NaN        NaN       NaN          NaN   
3          NaN       NaN        NaN        NaN       NaN          NaN   
4          NaN       NaN        NaN        NaN       NaN          NaN   
5          NaN       NaN        NaN        NaN       NaN          NaN   
6          NaN       NaN        NaN        NaN       NaN          NaN   
7          NaN       NaN        NaN        NaN       NaN          NaN   
8          NaN       NaN        NaN        NaN       NaN          NaN   
9          NaN       NaN        NaN        NaN       NaN          NaN   
10         NaN       NaN        NaN        NaN       NaN          NaN   
11         NaN       NaN   9.146583        NaN       NaN          NaN   
12         NaN       NaN   9.150633        NaN     

In [54]:
print(df_r)

          date       code   open         r
0     20170103  000001.SZ   9.11  0.098793
1     20170104  000001.SZ   9.15  0.054645
2     20170105  000001.SZ   9.17  0.032715
3     20170106  000001.SZ   9.17  0.032715
4     20170109  000001.SZ   9.13  0.076670
5     20170110  000001.SZ   9.15  0.054645
6     20170111  000001.SZ   9.14  0.065646
7     20170112  000001.SZ   9.13  0.076670
8     20170113  000001.SZ   9.14  0.065646
9     20170116  000001.SZ   9.15  0.054645
10    20170117  000001.SZ   9.12  0.087719
11    20170118  000001.SZ   9.14  0.065646
12    20170119  000001.SZ   9.15  0.054645
13    20170120  000001.SZ   9.17  0.032715
14    20170123  000001.SZ   9.22 -0.021692
15    20170124  000001.SZ   9.23 -0.032503
16    20170125  000001.SZ   9.27 -0.075512
17    20170126  000001.SZ   9.27 -0.075512
18    20170203  000001.SZ   9.34 -0.149893
19    20170206  000001.SZ   9.26 -0.064795
20    20170207  000001.SZ   9.31 -0.118153
21    20170208  000001.SZ   9.29 -0.096878
22    20170

In [65]:
print(df_all.columns)
df_all2 = df_all.merge(df.reset_index()[["date","code","high","low","close","vol","amt","adj_factor"]],on=["date","code"],how="inner")

Index(['date', 'code', 'ma12', 'ma26', 'dif', 'dea', 'macd', 'macd_pct',
       'macd_pct2', 'open', 'r'],
      dtype='object')


In [67]:
print(df_all2.shape,df_all2.columns)

(31896, 17) Index(['date', 'code', 'ma12', 'ma26', 'dif', 'dea', 'macd', 'macd_pct',
       'macd_pct2', 'open', 'r', 'high', 'low', 'close', 'vol', 'amt',
       'adj_factor'],
      dtype='object')
